In [134]:
import requests, json
from collections import OrderedDict
from operator import getitem
import time
import pandas as pd
from web3 import Web3
import warnings
import struct
import binascii

In [135]:
beacon_chain_contract = Web3.to_checksum_address('0x00000000219ab540356cbb839cbe05303d7705fa')
beacon_chain_deposit_event = '0x649bbc62d0e31342afea4e5cd82d4049e7e1ee912fc0889aa790803be39038c5'
api_key = 'AMBERDATA_API_KEY'

In [136]:
# Replace with your AMBERDATA API KEY
eth_node_url = f'https://rpc.web3api.io/api/v2?x-api-key={api_key}'

In [137]:
w3 = Web3(Web3.HTTPProvider(eth_node_url))

# Check if connected to Ethereum node
if w3.is_connected:
    print("Connected to Ethereum node")
else:
    print("Not connected to Ethereum node")
    exit(1)

Connected to Ethereum node


In [138]:
headers = {
    "accept": "application/json",
    "x-amberdata-blockchain-id": "ethereum-mainnet",
    "x-api-key": f'{api_key}'
}

In [139]:
#Fetch Contract ABI: 
abi_url = f'https://web3api.io/api/v2/contracts/{beacon_chain_contract}' 

In [140]:
#Fetching contract address ABI, formating and extracting ABI Only. Will use later for parsing logs. 
abi = requests.request("GET", url=abi_url, headers=headers)
abi = json.loads(abi.text)
abi = abi['payload']['abi']

In [141]:
#Create Web3 Contract Instance: 
myContract = w3.eth.contract(address=beacon_chain_contract, abi=abi)

In [142]:
# Helper function to convert bytes to interger. 
def endian_to_int(endian):
    little_endian_bytes = bytes.fromhex(endian)
    reversed_bytes = little_endian_bytes[::-1]
    integer_value = int.from_bytes(reversed_bytes, byteorder='big')
    return integer_value

In [153]:
#Empyt dict for storing returned data. 
collection = {}
id = 0
#For Pagination
pagination = 0
total_pages = 1000
paginationIncrement = 1

In [154]:
while pagination <= total_pages:
    #Fetch Contract Events Logs. 
    url = f'https://web3api.io/api/v2/addresses/{beacon_chain_contract}/logs?topic={beacon_chain_deposit_event}&page={pagination}&size=100'
    response = requests.request("GET", url, headers=headers)
    while response.status_code == 429:
        time.sleep(1)
        response = requests.request("GET", url, params=querystring)
    batch_list = json.loads(response.text)["payload"]['records']
    for item in batch_list:
            try: 
                id +=1
                collection[id] = {}
                collection[id]['timestamp'] = item["timestamp"]
                collection[id]['transaction_hash'] = item["transactionHash"]
                collection[id]['block_number'] = item['blockNumber']
                collection[id]['pub_key'] = (item['data'][6] + item['data'][7])[:96]
                collection[id]['withdrawal_credentials'] = item['data'][9]
                collection[id]['amount'] = endian_to_int(item['data'][11])
                collection[id]['deposit_index'] = endian_to_int(item['data'][17])
                collection[id]['signature'] = item['data'][13]+item['data'][14]+item['data'][15]
            except NameError:
                print(NameError)
            except:
                print("No Data")
    print('Finished page',pagination)
    pagination += paginationIncrement


Finished page 0
Finished page 1
Finished page 2
Finished page 3
Finished page 4
Finished page 5
Finished page 6
Finished page 7
Finished page 8
Finished page 9
Finished page 10
Finished page 11
Finished page 12
Finished page 13
Finished page 14
Finished page 15
Finished page 16
Finished page 17
Finished page 18
Finished page 19
Finished page 20
Finished page 21
Finished page 22
Finished page 23
Finished page 24
Finished page 25
Finished page 26
Finished page 27
Finished page 28
Finished page 29
Finished page 30
Finished page 31
Finished page 32
Finished page 33
Finished page 34
Finished page 35
Finished page 36
Finished page 37
Finished page 38
Finished page 39
Finished page 40
Finished page 41
Finished page 42
Finished page 43
Finished page 44
Finished page 45
Finished page 46
Finished page 47
Finished page 48
Finished page 49
Finished page 50
Finished page 51
Finished page 52
Finished page 53
Finished page 54
Finished page 55
Finished page 56
Finished page 57
Finished page 58
Finishe

In [155]:
#updating some pandas default settings
pd.set_option('max_colwidth', None) # show full width of showing cols
pd.set_option("expand_frame_repr", False) # print cols 
pd.set_option('display.float_format', lambda x: f'{x:.3f}') # Format number

In [157]:
df = pd.DataFrame.from_dict(collection,orient='index')

In [158]:
df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')

In [160]:
df

,timestamp,transaction_hash,block_number,pub_key,withdrawal_credentials,amount,deposit_index,signature,datetime
1,1694009843000,0xe11f9c9ff205e399942327286682c6845b548c6aad097054460dd5f88ab5e015,18077943,80dff79744c11ca0c087e7edbcc3d7e65984bc07162c8ad35bd0162f6174ce32c936aee59d1e446761dd5275ed7bb554,01000000000000000000000024814e2d11cdf985b70277550258a9cfbcf48f13,32000000000,949178,91772df77fa067d2f8dbec2bb8f011602a5c04288793a9773df0d9a75c644139424c1b226de0cc7d2f6f94d0f0b31737030645b32fb3c98c2e94fde094012f71ed44287a94e249aa1e4a25e6fad55a63bea4a47d0d5fc300740df88db6fd49e5,2023-09-06 14:17:23
2,1694009147000,0x4870840a7344a3db3522f05e48575e3171c21cefef57a881e4761164e24fca4b,18077885,8db8d18567ab969534140c52ad15932c391dbc45145f558d6d1343eba5be34249eb19711688ba8de8fd639076f22bc23,010000000000000000000000210b3cb99fa1de0a64085fa80e18c22fe4722a1b,32000000000,949177,84ecebcfaa792957d94c5d24d3a24bac0b9326b919a9e547a386a50a5d7fed5c7513e9dd50535701768082dbe5085108164e9e9a8fdcdf7568fda5c66d8e22f0f4c450dbcee6802d342d89f9cfbed35d5b777b53ceadeb5573f66029a97e8f8f,2023-09-06 14:05:47
3,1694009147000,0x4870840a7344a3db3522f05e48575e3171c21cefef57a881e4761164e24fca4b,18077885,b36a13dff4c7d39c444c66d7c87c3bb1933e69eb2f22be2b1fd468dda8432a22af35f07fde4f235b2d3bdea471ad9ffd,010000000000000000000000210b3cb99fa1de0a64085fa80e18c22fe4722a1b,32000000000,949176,b31e2c03777438325d401ddefd7e920d7349d25f31bfc4cf2f9887f8f5a3eb1aa2e1b73aafb6b3a66d1d3422382f39e70f697f6569f01d06ddac71f807d4875406a1939138092e9453dc0941e376bfb056e177a68f7ad3e9d1c8def1247716a1,2023-09-06 14:05:47
4,1694009147000,0x4870840a7344a3db3522f05e48575e3171c21cefef57a881e4761164e24fca4b,18077885,9913aefdef8731bf68ee7bc64f3b3911ae0436e580dbd8e9b2e3523348bb46aa837d96502039b8d90dbb111a89fcef08,010000000000000000000000210b3cb99fa1de0a64085fa80e18c22fe4722a1b,32000000000,949175,867aa5f37f2c69ca14e6b7e1e362e66bc56222022ddef76d5d2c676c3b196e00b78fa910fe998dd230245004f07cefba064b26e94ea61038f799ce79c245e039f578e854804b3a56c714f528a01445c6ef899ad5123d22fa2de8ce551e8178d5,2023-09-06 14:05:47
5,1694009147000,0x4870840a7344a3db3522f05e48575e3171c21cefef57a881e4761164e24fca4b,18077885,aeb8e70e72681fec084653e5d6995e20ef511057b4ae257effe11891c6b78938de56ef0a26f7df0a4da729b0d964bf59,010000000000000000000000210b3cb99fa1de0a64085fa80e18c22fe4722a1b,32000000000,949174,b56d6ad59b89e42e9c8ee4cb742294232858b77ea81e4c177ab7b0a91d467d3e2087b35f5b19217d6087512689f8a5cd0acfb3c5a8fa228f70f894a139ba87e318a2f0036f0ecb7d804b9c73bf2cbcca14e209e7d81b7baea95ae96d7bbc5dce,2023-09-06 14:05:47
...,...,...,...,...,...,...,...,...,...
100096,1689127187000,0x6bab2cff80f922b1456b057b3be6e55e68ba01c3a7fffd62d2a409a77583a3d2,17674382,b906de7e073d27fdec9021353f53c2b82650ad783d204541985248a3d76e287b75b89c3bafabb65ea4550515afaa9aba,010000000000000000000000f20b338752976878754518183873602902360704,32000000000,849083,b5ec91a12fb444fd0cf5c0edadc58b71f18d35b499a876d8836941d57b29c2d4df66a383b3b99e8044a3e9785f6857e4194e4dea31aae46fed20ec99ed2110f457176fc5bec6cc4e12d727022065da5621a33dace839776f8ef80f5c8388bf97,2023-07-12 01:59:47
100097,1689127187000,0x6bab2cff80f922b1456b057b3be6e55e68ba01c3a7fffd62d2a409a77583a3d2,17674382,842c62977234ded14b90f001afe492f1daa3014647c077b22175392063c96dde9963cc3b92a3722b7d1a81481d0943a9,010000000000000000000000f20b338752976878754518183873602902360704,32000000000,849082,81edbd3c3c76d86995c52f0caff3d7abf1f8ab4a1d4ef979f799d8a93098a8287b053303f82fa26d558f00cf0f694199125a2e3fab1ae017b73bc9bf04f99dc8347967806c9798316c8caab2fad77f4a6c1c2109b9a5198c37a8a74338c3abdc,2023-07-12 01:59:47
100098,1689127187000,0x6bab2cff80f922b1456b057b3be6e55e68ba01c3a7fffd62d2a409a77583a3d2,17674382,ab51b1571b94aea65766b7df0f32cdf1488f8b31c0fefc54981ef2f2618a57abcf4de12c37afe3db48d35e3c1b9254a2,010000000000000000000000f20b338752976878754518183873602902360704,32000000000,849081,a32c9b44aede6a08fadce683f03684276238b6942e0e36929f7c33c1f23f7a509546a98f9a12fdc51dbd1a56097c15d1199e727aae824aee2cb5ede5a052779a4d7b5e41627a9f3a837b54b600bad6

In [163]:
#Alternative Approach is using the ABI + Web3.py to Process_Receipt - This approach is a bit slower since it request an RPC call. 
# while pagination <= total_pages:
#     #Fetch Contract Events Logs. 
#     url = f'https://web3api.io/api/v2/addresses/{beacon_chain_contract}/logs?topic={beacon_chain_deposit_event}&page={pagination}&size=100'
#     response = requests.request("GET", url, headers=headers)
#     while response.status_code == 429:
#         time.sleep(1)
#         response = requests.request("GET", url, params=querystring)
#     batch_list = json.loads(response.text)["payload"]['records']
#     for item in batch_list:
#         try: 
#             print(item["transactionHash"])
#             receipt = w3.eth.get_transaction_receipt(item["transactionHash"])
#             logs = myContract.events.DepositEvent().process_receipt(receipt)
#             id +=1
#             collection[id] = {}
#             collection[id]['timestamp'] = item["timestamp"]
#             collection[id]['transaction_hash'] = item["transactionHash"]
#             collection[id]['blockNumber'] = logs[0]['blockNumber']
#             collection[id]['amount_raw'] = logs[0]['args']['amount']
#             collection[id]['amount_clean'] = endian_to_int(logs[0]['args']['amount'])
#             collection[id]['index'] = logs[0]['args']['index']
#             collection[id]['signature'] = logs[0]['args']['signature']
#             collection[id]['signature_cleaned'] = binascii.hexlify(logs[0]['args']['signature']).decode('utf-8')
#             collection[id]['withdrawal_credentials'] = logs[0]['args']['withdrawal_credentials']
#             collection[id]['withdrawal_credentials_cleaned'] = binascii.hexlify(logs[0]['args']['withdrawal_credentials']).decode('utf-8')
#         except NameError:
#             print(NameError)
#         except:
#             print("No Events at this TX")
#     print('Finished page',pagination)
#     pagination += paginationIncrement
